In [3]:
import pandas as pd
import glob
from googletrans import Translator
from google_trans_new import google_translator  
import emoji
import time
import re
from langdetect import detect
import demoji
import fasttext
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [4]:
directory = 'targeted'
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)

In [5]:
nltk.download('wordnet')
#nltk.download() # only needs to be ran once
wnl = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/BrianPark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/BrianPark/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F680-\U0001F6FF"  # CJK Unified Ideographs Extension B
                           u"\U0001F600-\U0001F64F"  # CJK Unified Ideographs
                           u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs"
                           u"\U0001F000-\U0001F02F"  # Mahjong Tiles
                           u"\U0001F004-\U0001F0CF"  # Domino Tiles
                           u"\U0001F110-\U0001F251"  # Playing Cards
                           u"\U0001F600-\U0001F64F"  # Emoticons (faces)
                           u"\U0001F910-\U0001F91F"  # Emojis (animal faces)
                           u"\U0001F920-\U0001F927"  # Emojis (human faces)
                           u"\U0001F930-\U0001F939"  # Emojis (clothing)
                           u"\U0001F950-\U0001F95F"  # Emojis (food)
                           u"\U0001F980-\U0001F991"  # Emojis (objects)
                           u"\U0001F9C0-\U0001F9C0"  # Emojis (symbols)
                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)



In [7]:
t = Translator()

translations = []
test_file = glob.glob(directory + "/*.xlsx")

test_list = []
for file in test_file:
    df = pd.read_excel(file, sheet_name='Tweets')
    test_list.append(df)


# Concatenate the DataFrames in test_list into a single DataFrame
combined_df = pd.concat(test_list, axis=0, ignore_index=True)
combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)
combined_df['Tweet'] = combined_df['Tweet'].str.replace("’", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‘", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("“", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("”", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‼️", "!!")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("…", "...")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("⁃", "-")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("☹️", "")
combined_df['Tweet'] = combined_df['Tweet'].apply(remove_emojis)



language_model = fasttext.load_model('lid.176.bin') 

#initalize already to English
combined_df['Is_English'] = '1'

for index, row in combined_df.iterrows():
    text = row['Tweet']
    text = text.replace('\n', ' ')
    try:
        prediction = language_model.predict(text)
        detected_language = prediction[0][0].replace("__label__", "")
        
        # Define a confidence threshold to consider a language as detected
        confidence_threshold = 0.8 
        if prediction[1][0] >= confidence_threshold:
            if detected_language == 'en':
                combined_df.at[index, 'Is_English'] = '1'  # English
            else:
                combined_df.at[index, 'Is_English'] = '0'  # Non-English
    except Exception as e:
        print(f"Error detecting language: {e}")

mask = combined_df['Is_English'].str.contains('0', case=False)


# Use the mask to filter rows that match the condition
filtered_rows = combined_df[mask]

filtered_rows.to_csv('filtered.csv')

df = pd.read_csv('filtered.csv')
# Extract the 'Tweet' column
texts = df['Tweet'].tolist()

batch_size = 10  

translations = []

for i in range(0, len(texts), batch_size):
    text_batch = texts[i:i + batch_size]
    translated_batch = []

    for text in text_batch:
        try:
            translated_text = t.translate(text, dest='en').text
            if translated_text is not None:
                translated_batch.append(translated_text)
            else:
                translated_batch.append("")
            time.sleep(1)
        except Exception as e:
            print(f"Error translating: {e}")
            translated_batch.append("Wrong") # Placeholder to determine which non-English text is causing issues
    translations.extend(translated_batch)

df['Translated'] = translations




In [8]:
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up the jewelry, came home and knocked four chains, Meghaninagar case\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation."
...,...,...,...,...,...,...,...,...
1994,458789,B V Prasad Reddy,@bvprasadreddy,2023-01-20 01:31:51,"నీచులు\nసమాజంలో\nచెడు నీచం పెంచడం కోసం,\nమంచిని తగ్గించడం కోసం&amp;\nTargeted personsను ఇబ్బంది పెట్టడం కోసం\nఒక vaccum, situations things create చేసి పెడితే\nనాయకులు అనే వారు doubt discuss debate చేసి\nఅసలెందుకిదంతా అని\nమంచి చెడు విచక్షణ\nవిషయ పరిజ్ఞానం\nఇంగితజ్ఞానంతో\nఆపాలి గానీ\nనీచుల మాయలో",https://www.twitter.com/user/status/1616247098974040064,0,"The miseries\nIn society\nFor raising the bad miserable,\nFor reducing the good & amp;\nFor embarrassing targeted persons\nIf a vaccum, SITUATIONS THINGS CREATE\nLeaders Doubt Discusss Debate\nAll right\nGood bad discretion\nSubject knowledge\nWith common sense\nStop\nIn the magic of the nihils"
1995,458852,🐨❤🍯🧸🌸🍡🌈🌟,@eiLoJDaPKooL,2023-01-20 01:23:00,"RT @inspoPageant: HIT THE POINT/ \n\nในรอบ closed-door interview #MissUniverseThailand2022 จากคำตอบของ R'Bonney Nola #MissUniverse2022 น่าจะทำให้น้อง แอนชิลี ได้รับคำตอบที่เคยถาม นิโคลีน แล้วนะว่า \n\nเพราะเหตุใดถึงต้องตอบในแบบ Beauty Queen ถึงจะมง \n\n""Get to the point, hit t

In [9]:
# Some texts do not end up translating to English
df2 = df[df['Translated'].str.contains("Wrong")]
df2['Tweet'] = df2['Tweet'].str.replace("☹️", "")
df2.loc[:, 'Tweet'] = df2['Tweet'].apply(remove_emojis) #Emojis not removed for some reason
for index, row in df2.iterrows():
    text = row['Tweet']
    try:
        translated = t.translate(text, dest='en')  
        if translated:
            translated_text = translated.text
            df2.loc[index, 'Translated'] = translated_text
        else:
            df2.loc[index, 'Translated'] = "Translation Failed"
    except Exception as e:
        print(f"Error translating: {e}")
        df2.loc[index, 'Translated'] = "Translation Error"


In [10]:
df2

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated


In [11]:
# delete placeholders and add translated texts to originial dataframe

df = df[~df['Translated'].str.contains('Wrong')]
df = pd.concat([df, df2], axis=0, ignore_index=True)
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up the jewelry, came home and knocked four chains, Meghaninagar case\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation."
...,...,...,...,...,...,...,...,...
1994,458789,B V Prasad Reddy,@bvprasadreddy,2023-01-20 01:31:51,"నీచులు\nసమాజంలో\nచెడు నీచం పెంచడం కోసం,\nమంచిని తగ్గించడం కోసం&amp;\nTargeted personsను ఇబ్బంది పెట్టడం కోసం\nఒక vaccum, situations things create చేసి పెడితే\nనాయకులు అనే వారు doubt discuss debate చేసి\nఅసలెందుకిదంతా అని\nమంచి చెడు విచక్షణ\nవిషయ పరిజ్ఞానం\nఇంగితజ్ఞానంతో\nఆపాలి గానీ\nనీచుల మాయలో",https://www.twitter.com/user/status/1616247098974040064,0,"The miseries\nIn society\nFor raising the bad miserable,\nFor reducing the good & amp;\nFor embarrassing targeted persons\nIf a vaccum, SITUATIONS THINGS CREATE\nLeaders Doubt Discusss Debate\nAll right\nGood bad discretion\nSubject knowledge\nWith common sense\nStop\nIn the magic of the nihils"
1995,458852,🐨❤🍯🧸🌸🍡🌈🌟,@eiLoJDaPKooL,2023-01-20 01:23:00,"RT @inspoPageant: HIT THE POINT/ \n\nในรอบ closed-door interview #MissUniverseThailand2022 จากคำตอบของ R'Bonney Nola #MissUniverse2022 น่าจะทำให้น้อง แอนชิลี ได้รับคำตอบที่เคยถาม นิโคลีน แล้วนะว่า \n\nเพราะเหตุใดถึงต้องตอบในแบบ Beauty Queen ถึงจะมง \n\n""Get to the point, hit t

In [12]:
lemmatizer = WordNetLemmatizer()

def lemmatize_with_pos_tag(sentence):
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:
            return None

    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))

    wordnet_tagged = [(word, pos_tagger(tag)) for word, tag in pos_tagged]

    lemmatized_tokens = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:
            # else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
    
    lemmatized_sentence = " ".join(lemmatized_tokens)
    return lemmatized_sentence

df['complex_lemmatized_column'] = df['Translated'].apply(lambda x: lemmatize_with_pos_tag(x))

In [13]:
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated,complex_lemmatized_column
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO","@ dom_vom_gym feel Giga Targeted after I be at the doctor today , but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k","RT @ spectrum : to target cancer cell in a targeted manner , increase the chance of recovery enormously.With `` Targeted Therapy '' , doctor can hit certain tumor so precisely that the rest of the body be largely spared. # Krebs # weltkrebstag @ dkfz @ xGeKrebs http : //t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k","RT @ spectrum : to target cancer cell in a targeted manner , increase the chance of recovery enormously.With `` Targeted Therapy '' , doctor can hit certain tumor so precisely that the rest of the body be largely spared. # Krebs # weltkrebstag @ dkfz @ xGeKrebs http : //t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up the jewelry, came home and knocked four chains, Meghaninagar case\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey","Speak Lyo .. Since go to the wedding , the jewelry go to pick up the jewelry , come home and knock four chain , Meghaninagar case # Ahmedabad # vtvgujarati http : //t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation.","@ edelman_vincent @ cbokhove @ researched_be @ researched_NL In my experience it be a nuanced process.Roughly 'targeted ' for remember and give information about learning , 'holistic ' for con

In [14]:
eng_df = combined_df[combined_df['Is_English'].str.contains('1', case=False)]
eng_df['complex_lemmatized_column'] = eng_df['Tweet'].apply(lambda x: lemmatize_with_pos_tag(x))

/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_30242/4234526540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_df['complex_lemmatized_column'] = eng_df['Tweet'].apply(lambda x: lemmatize_with_pos_tag(x))


In [15]:
eng_df

,User,Username,UTC Date,Tweet,Permalink,Is_English,complex_lemmatized_column
0,Gary Hill,@hillg367,2023-01-30 19:39:18,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144641940258816,1,"RT @ davenewworld_2 : 4 white supremacist in Washington state be sentence to 28 to 48 month for a hate crime after they enter a bar and assault the DJ , Tyrone Smith , while call him slur . They target a Black man on `` Martyr 's Day '' to `` honor '' a white supremacist who die in the 1980s . http : //t.co/O3ojTWDpVL"
1,خالد الشعيبي,@qwe716128968,2023-01-30 19:39:16,The US directly targeted all economic sectors in #Yemen and looted all the revenues leaving the state employees with nothing to live off of\n#USAEnemyOfPeace\n#امريكا_عدو_السلام,https://www.twitter.com/user/status/1620144636416380928,1,The US directly target all economic sector in # Yemen and loot all the revenue leave the state employee with nothing to live off of # USAEnemyOfPeace # امريكا_عدو_السلام
2,ampk.95,@ampk_95,2023-01-30 19:39:16,@mccnshyne @akitoshinonome Not really an otome at all. More just like you learn lore about the characters and listen to all the songs they got released. Theres a game where you can collect their cards. Technically targeted for women but like you just kinda hang out with them as they info dump their issues,https://www.twitter.com/user/status/1620144635808223233,1,@ mccnshyne @ akitoshinonome Not really an otome at all . More just like you learn lore about the character and listen to all the song they get release . Theres a game where you can collect their card . Technically target for woman but like you just kinda hang out with them as they info dump their issue
3,عبدالله,@bdllh16716948,2023-01-30 19:39:15,Every aspect of life in #Yemen was targeted by the US-Saudi aggression even chicken farms and horse stables were not left untouched\n#USAEnemyOfPeace\n#امريكا_عدو_السلام\nhttps://t.co/ieboNZTpfD,https://www.twitter.com/user/status/1620144629135069184,1,Every aspect of life in # Yemen be target by the US-Saudi aggression even chicken farm and horse stable be not leave untouched # USAEnemyOfPeace # امريكا_عدو_السلام http : //t.co/ieboNZTpfD
4,llwry,@LLLwry,2023-01-30 19:39:14,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144627256025088,1,"RT @ davenewworld_2 : 4 white supremacist in Washington state be sentence to 28 to 48 month for a hate crime after they enter a bar and assault the DJ , Tyrone Smith , while call him slur . They target a Black man on `` Martyr 's Day '' to `` honor '' a white supremacist who die in the 1980s . http : //t.co/O3ojTWDpVL"
...,...,...,...,...,...,...,...
459854,Jamie Chirps 🦜🇨🇦,@JamieLC6,2023-01-19 23:21:16,RT @Roman_Baber: ICYMI. The same Canadian Armed Forces team that engaged in propaganda against Canadians at the start of the Pandemic has targeted Canadians again. \n\n@AnitaAnandMP &amp; @JustinTrudeau are spreading propaganda and using the @CanadianForces against Canadians. https://t.co/doJZQXCdns,https://www.twitter.com/user/status/1616214235918356481,1,RT @ Roman_Baber : ICYMI . The same Canadian Armed Forces team that engage in propaganda against Canadians at the start of the Pandemic have target Canadians again . @ AnitaAnandMP & amp ; @ JustinTrudeau be spread propaganda and use the @ CanadianForces against Canadians . http : //t.co/doJZQXCdns
459855,UnacceptableSilverFox 🍁,@Cdn_

In [17]:
merged_df = pd.concat([eng_df, df], axis=0, ignore_index=True)
merged_df
merged_df.to_csv('merged_df.csv', index = False)
